In [11]:
import numpy as np
import pandas as pd

import os
import random

### Загрузка данных

In [9]:
path_data = './data'

data1 = []
data2 = []

# __ считываем первый сплит __
with open(os.path.join(path_data, 'c1.txt')) as c1:
    for line in c1:
        clicks = line.strip().split()
        data1.append(np.array(clicks).astype(int))
data1 = np.asarray(data1)

# __ считываем второй сплит __
with open(os.path.join(path_data, 'c2.txt')) as c2:
    for line in c2:
        clicks = line.strip().split()
        data2.append(np.array(clicks).astype(int))
data2 = np.asarray(data2)

In [10]:
print data1[:5]

[[1 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [1 1 1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]]


### Bootstrap

In [59]:
# __ Получение подвыборки того же размера __
def get_sample(data):
    n = len(data)
    new_data = []
    for i in xrange(n):
        new_data.append(data[random.randint(0, n-1)])
    
    return np.asarray(new_data)

In [68]:
def t_stats(data1, data2):
    return abs(data1.mean() - data2.mean()) / np.sqrt(data1.std() / len(data1) + data2.std() / len(data2))

In [72]:
def isReject(t, t0=1.96):
    return int(t > t0)

In [84]:
def bootstrap(data1, data2, stats, verbose=False):
    new_data1 = get_sample(data1)
    new_data2 = get_sample(data2)
    t = t_stats(stats(new_data1), stats(new_data2))
    if verbose:
        print 't = ' + str(t)
    return isReject(t)

In [85]:
def ASL(data1, data2, stats, n_bootstrap=5000, verbose=False):
    
    n_reject = 0
    for i in xrange(n_bootstrap):
        n_reject += bootstrap(data1, data2, stats, verbose)

    return float(n_reject) / n_bootstrap

### CTR

In [57]:
def CTR(data):
    no_click = [1 if (x==0).all() else 0 for x in data]
    return no_click.count(0) / float(len(data))

In [58]:
# __ Получаем стипок значений CTR для каждой группы __ 
def all_CTR(data, len_group=1000):
    result = []
    for i in xrange(len(data) / len_group):
        result.append(CTR(data[i*len_group : (i+1)*len_group]))
        
    return np.asarray(result)

In [80]:
print 'ASL: ' + str(ASL(data1, data2, all_CTR, n_bootstrap=10, verbose=True))

t = 5.25899022208
t = 5.52319056013
t = 5.31216238201
t = 5.59531523695
t = 5.62369504629
t = 5.35236965265
t = 5.55305719367
t = 5.67220571617
t = 5.4530701701
t = 5.5676105759
ASL: 1.0


In [87]:
print 'Количество итераций: 10.000'
print 'ASL: ' + str(ASL(data1, data2, all_CTR, n_bootstrap=10000, verbose=False))

Количество итераций: 10.000
ASL: 1.0


Отвергается гипотеза Н0 о том, что CTR на двух сплитах можно объяснить шумами

### Доля кликов в первый результат

In [81]:
def firstClick(data):
    first_click = [1 if x[0]==1 else 0 for x in data]
    return first_click.count(1) / float(len(data))

In [82]:
def all_firstClick(data, len_group=1000):
    result = []
    for i in xrange(len(data) / len_group):
        result.append(firstClick(data[i*len_group : (i+1)*len_group]))
        
    return np.asarray(result)

In [83]:
print 'ASL: ' + str(ASL(data1, data2, all_firstClick, n_bootstrap=10, verbose=True))

t = 27.5055752207
t = 28.1929446624
t = 27.8416280869
t = 28.0082916252
t = 28.1206978403
t = 28.2285347774
t = 28.0607517875
t = 28.2134900879
t = 28.0246417723
t = 27.6358489501
ASL: 1.0


In [88]:
print 'Количество итераций: 10.000'
print 'ASL: ' + str(ASL(data1, data2, all_firstClick, n_bootstrap=10000))

Количество итераций: 10.000
ASL: 1.0


Отвергается гипотеза Н0 о том, что доля кликов в первый результат на двух сплитах можно объяснить шумами